# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your https://hub.ischool.syr.edu/ workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [1]:
# import packages
import numpy as np
import matplotlib.pyplot as plt

# Part 3: Optimization

In one of the activites in class, we saw how to find the optimal coefficients of a linear regression model using the equation

$$
\hat{\beta} = (X^T X)^{-1} X^T y
$$

which is the result of finding the best parameters that minimize the loss
$$
L(\beta) = \sum_i (\hat{y}_i - y_i)^2
$$

where 
$$
y_i = b_0 + \sum_i b_j x_j
$$

Sometimes, however, we want to minimize the following _regularized_ loss function known as ridge loss

$$
L(\beta) = \sum_i (\hat{y}_i - y_i)^2 + \gamma \sum_j b_j^2
$$

where $\gamma$ is non-negative real number. This loss function can be solved with

$$
\hat{\beta} = (X^T X + \gamma I)^{-1} X^T y
$$

where $I$ is an identity matrix with the same dimensions as $X^T X$.


For the given random dataset:

In [2]:
import numpy as np
# the following code generates 1000 data points with 4 features each, and you are trying to predict one outcome y
# this makes the results reproducible
np.random.seed(42)
# generate X and Y
true_beta = np.random.random(size=(2, 1))
X = np.random.random(size=(1000, 2))
y = X @ true_beta + 1/8*np.random.randn(1000, 1)

**Question 3.1 (5 pts)** Create a `find_ols` function that for a given `X` and `y` Numpy arrays returns the optimal coefficients for squared errors. Remember that matrix inversion can be computed with the `np.linalg.inv` function.

In [3]:
def find_ols(X, y):
    # YOUR CODE HERE
    #raise NotImplementedError()
    b = np.linalg.inv(X.T @ X) @ X.T @ y
    return b

In [4]:
# check your code. The solution should be closer to the true beta
print("True solution", true_beta)
print("Estimation", find_ols(X, y))

True solution [[0.37454012]
 [0.95071431]]
Estimation [[0.37394758]
 [0.95054005]]


In [5]:
# 5 pts
np.testing.assert_almost_equal(find_ols(X, y), np.array([[0.37394758],
       [0.95054005]]))
# solution size
np.testing.assert_equal(find_ols(np.random.random(size=(100,10)), np.random.random(size=(100,1))).shape, (10, 1))

**Question 3.2 (5 pts)** Define a function `SS` that receives Numpy arrays `X`, `y`, `b` and computes the sum of squared errors using Numpy.

$$
SS = \sum_i (\sum_j b_j x_j - y_i)^2
$$

which you can represent with

$$
SS = (X \hat{\beta} - y)^T (X \hat{\beta} - y)
$$

**Note that your function should return just a number and not a matrix**

In [6]:
def SS(X, y, b):
    return (((X@b-y).T)@(X@b-y))[0][0]

    #raise NotImplementedError()

In [7]:
# try it here
SS(X, y, find_ols(X, y))

14.971442060168258

In [8]:
# 5 pts
np.testing.assert_equal(SS(X, y, find_ols(X, y)), 14.971442060168256)

AssertionError: 
Items are not equal:
 ACTUAL: 14.971442060168258
 DESIRED: 14.971442060168256

**Question 3.3 (5 pts)** Define a function `find_ridge` that receives `X`, `y`, and `gamma` and returns the solution to the ridge loss. Use Numpy

In [12]:
def find_ridge(X, y, gamma):
    p = X.T@X
    xr,xc = p.shape
    c= np.linalg.inv(((X.T)@X)+ (gamma*np.eye(xr))) @(X.T)@y
    return c
    # YOUR CODE HERE
    #raise NotImplementedError()

By definition, the solution should be the same when $\gamma = 0$

In [13]:
print(find_ols(X, y))
print(find_ridge(X, y, 0))

[[0.37394758]
 [0.95054005]]
[[0.37394758]
 [0.95054005]]


With ridge regression, the coefficients should decrease with bigger values of gamma

In [14]:
for gamma in (1, 10, 100, 1000):
    print('gamma:', gamma, find_ridge(X, y, gamma))

gamma: 1 [[0.37612875]
 [0.94614896]]
gamma: 10 [[0.39279017]
 [0.90987695]]
gamma: 100 [[0.43118467]
 [0.70158231]]
gamma: 1000 [[0.2199223]
 [0.2692352]]


With ridge regression, the sum of squares should increase, making the solution _worse_ for the training data

In [15]:
for gamma in (1, 10, 100, 1000):
    print('MSE:', gamma, SS(X, y, find_ridge(X, y, gamma)))

MSE: 1 14.97477628605608
MSE: 10 15.267414386599514
MSE: 100 29.969901275064654
MSE: 1000 232.2762862209032


In [16]:
# 5 pts
np.testing.assert_almost_equal(find_ols(X, y), find_ridge(X, y, 0))
# solution size
np.testing.assert_array_less(find_ridge(X, y, 1000)**2, find_ridge(X, y, 5)**2)